In [159]:
!pip install xgboost catboost

In [185]:
from pathlib import Path

import joblib
import os
import numpy as np
import pandas as pd
import sklearn.gaussian_process.kernels as kernels

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge
from sklearn.svm import NuSVR
from joblib import Memory
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score

In [161]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [162]:
BASE_PATH = Path("/content/gdrive/MyDrive/data")
x_train = pd.read_csv(BASE_PATH / 'X_train.csv', skiprows=1, header=None).values[:, 1:]
x_test = pd.read_csv(BASE_PATH / 'X_test.csv', skiprows=1, header=None).values[:, 1:]
y_train = pd.read_csv(BASE_PATH / 'y_train.csv', skiprows=1, header=None).values[:, 1:].flatten()

In [163]:
def remove_outliers(x_train, y_train, x_test, contamination=0.047, variance_threshold=1e-7, random_state=42):
    """
    Remove low-variance features and outliers from training data.

    Parameters:
    -----------
    x_train : DataFrame or ndarray
    y_train : Series or ndarray
    x_test : DataFrame or ndarray
    contamination : float, proportion of outliers
    variance_threshold : float, threshold for variance filtering
    random_state : int

    Returns:
    --------
    x_train_clean, y_train_clean, x_test_clean
    """

    # Convert to DataFrame if needed
    x_train_df = pd.DataFrame(x_train).copy()
    x_test_df = pd.DataFrame(x_test).copy() if x_test is not None else None
    y_train_series = pd.Series(np.asarray(y_train).flatten(), index=x_train_df.index)

    # ===== Step 1: Remove zero/low-variance features =====
    var_selector = VarianceThreshold(threshold=variance_threshold)

    # Fit on training data (using median-imputed values to handle NaN)
    med = x_train_df.median(axis=0)
    x_train_for_var = x_train_df.fillna(med)

    var_selector.fit(x_train_for_var)

    variance_mask = var_selector.get_support()
    n_removed = (~variance_mask).sum()
    print(f"[VarianceThreshold] Removed {n_removed} low-variance features (threshold={variance_threshold})")

    x_train_df = x_train_df.iloc[:, variance_mask]
    if x_test_df is not None:
        x_test_df = x_test_df.iloc[:, variance_mask]

    med = x_train_df.median(axis=0)
    xtr_imp = x_train_df.fillna(med)

    scaler = RobustScaler()
    xtr_std = scaler.fit_transform(xtr_imp)

    pca = PCA(n_components=2, random_state=random_state)
    x_proj = pca.fit_transform(xtr_std)

    iso = IsolationForest(contamination=contamination, random_state=random_state)
    mask = iso.fit_predict(x_proj) == 1

    outlier_pos = np.where(~mask)[0]
    n_outliers = outlier_pos.size
    print(f"[OutlierRemoval] Removed {n_outliers} outliers ({n_outliers/len(mask)*100:.2f}%)")
    print(f"[Outlier positions] {outlier_pos.tolist()}")

    x_train_clean = x_train_df[mask]
    y_train_clean = y_train_series[mask]

    if isinstance(x_train, np.ndarray):
        return x_train_clean.values, y_train_clean.values, x_test_df.values if x_test_df is not None else None
    else:
        return x_train_clean, y_train_clean, x_test_df

In [164]:
class ScaledKNNImputer(BaseEstimator, TransformerMixin):
    """
    KNN Imputer that internally scales data for distance calculation,
    then returns imputed data in the ORIGINAL scale.

    This avoids having scaling artifacts in your pipeline.
    """

    def __init__(self, n_neighbors=7, weights='distance'):
        self.n_neighbors = n_neighbors
        self.weights = weights

    def fit(self, X, y=None):
        X_df = pd.DataFrame(X)

        self.median_ = X_df.median(axis=0)
        X_filled = X_df.fillna(self.median_)

        self.scaler_ = StandardScaler()
        self.scaler_.fit(X_filled)

        X_scaled = self.scaler_.transform(X_filled)
        self.imputer_ = KNNImputer(n_neighbors=self.n_neighbors, weights=self.weights)
        self.imputer_.fit(X_scaled)

        return self

    def transform(self, X):
        X_df = pd.DataFrame(X)

        X_filled = X_df.fillna(self.median_)

        X_scaled = self.scaler_.transform(X_filled)

        X_imputed_scaled = self.imputer_.transform(X_scaled)

        X_imputed_original = self.scaler_.inverse_transform(X_imputed_scaled)

        return X_imputed_original

In [165]:
class SpearmanRFSelector(BaseEstimator, TransformerMixin):
    """
    Combined feature selector:
    1. Scale data (using provided scaler)
    2. Select top_k features by Spearman correlation
    3. Select top_k features by RandomForest importance

    Works on already-imputed data.
    """

    def __init__(
        self,
        scaler=None,
        feature_tops=(200, 200),  # (spearman_top_k, rf_top_k) as tuple
        rf_n_estimators=1000,
        rf_max_depth=None,
        rf_min_samples_leaf=1,
        rf_max_features='sqrt',
        random_state=42
    ):
        self.scaler = scaler
        self.feature_tops = feature_tops
        self.rf_n_estimators = rf_n_estimators
        self.rf_max_depth = rf_max_depth
        self.rf_min_samples_leaf = rf_min_samples_leaf
        self.rf_max_features = rf_max_features
        self.random_state = random_state

    def fit(self, X, y):
        if y is None:
            raise ValueError("SpearmanRFSelector requires y")

        spearman_top_k, rf_top_k = self.feature_tops

        X_df = pd.DataFrame(X).copy() if not isinstance(X, pd.DataFrame) else X.copy()
        y_series = pd.Series(np.asarray(y).flatten(), index=X_df.index)

        self.n_features_in_ = X_df.shape[1]

        if self.scaler is not None:
            self.scaler_ = clone(self.scaler)
            X_scaled = self.scaler_.fit_transform(X_df)
            X_scaled_df = pd.DataFrame(X_scaled, columns=X_df.columns, index=X_df.index)
        else:
            X_scaled_df = X_df

        spearman_corr = X_scaled_df.corrwith(y_series, method='spearman').abs()

        n_keep_spearman = min(spearman_top_k, len(spearman_corr))
        spearman_features = spearman_corr.nlargest(n_keep_spearman).index

        print(f"[SpearmanRFSelector] Spearman: Selected {len(spearman_features)} features (from {self.n_features_in_})")

        X_spearman = X_scaled_df.loc[:, spearman_features]

        self.rf_ = RandomForestRegressor(
            n_estimators=self.rf_n_estimators,
            max_depth=self.rf_max_depth,
            min_samples_leaf=self.rf_min_samples_leaf,
            max_features=self.rf_max_features,
            random_state=self.random_state,
            n_jobs=-1,
            verbose=0
        )
        self.rf_.fit(X_spearman, y_series)

        importances = pd.Series(self.rf_.feature_importances_, index=X_spearman.columns)
        importances = importances.sort_values(ascending=False)

        n_keep_rf = min(rf_top_k, len(importances)) if rf_top_k else len(importances)
        rf_features = importances.head(n_keep_rf).index

        print(f"[SpearmanRFSelector] RF: Selected {len(rf_features)} features (from {len(spearman_features)})")

        self.selected_features_ = [X_df.columns.get_loc(col) for col in rf_features]
        self.selected_feature_names_ = rf_features.tolist()

        print(f"[SpearmanRFSelector] FINAL: {len(self.selected_features_)} features")

        return self

    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        # Return selected features
        return X_df.iloc[:, self.selected_features_].values


In [166]:
def build_imputation_pipeline(random_state=42):
    pipeline = Pipeline(
        [
              ("knn_imputer", ScaledKNNImputer()),
            # ("iterative_imputer", IterativeImputer()),
        ]
    )
    return pipeline

In [167]:
def build_feature_selection_pipeline(random_state=42):

    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            random_state=random_state
        ))
    ])

In [168]:
def build_svr_branch(random_state=42):
    """SVR branch with feature selection"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", StandardScaler()),
        ("model", NuSVR())
    ])


def build_hgb_branch(random_state=42):
    """HistGradientBoosting with feature selection"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", StandardScaler()),
        ("model", HistGradientBoostingRegressor(random_state=random_state))
    ])

def build_xgb_branch(random_state=42):
    """XGBoost"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", RobustScaler()),
        ("model", XGBRegressor(
            random_state=random_state,
            n_jobs=-1,
            verbosity=0  # Suppress output
        ))
    ])


def build_cat_branch(random_state=42):
    """CatBoost"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", RobustScaler()),
        ("model", CatBoostRegressor(
            random_state=random_state,
            verbose=False,
            thread_count=-1
        ))
    ])


def build_etr_branch(random_state=42):
    """ExtraTrees with feature selection"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", StandardScaler()),
        ("model", ExtraTreesRegressor(random_state=random_state, n_jobs=-1))
    ])

def build_gpr_branch(random_state=42):
    """GaussianProcess with feature selection"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", RobustScaler()),
        ("model", GaussianProcessRegressor(random_state=random_state))
    ])

# TODO try with ensemble
def build_ridge_branch(random_state=42):
    """Ridge - Simple linear model"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", StandardScaler()),
        ("model", Ridge())
    ])

# TODO try with ensemble
def build_pca_ridge_branch(random_state=42):
    """Ridge with PCA instead of SpearmanRF selection"""
    return Pipeline([
        ("feature_selector", PCA(n_components=100, random_state=random_state)),  # Different approach!
        ("scaler", StandardScaler()),
        ("model", Ridge())
    ])


# TODO try with ensemble
def build_mlp_branch(random_state=42):
    """MLP - Multi-layer Perceptron (Neural Network)"""
    return Pipeline([
        ("feature_selector", SpearmanRFSelector(
            scaler=StandardScaler(),
            random_state=random_state
        )),
        ("scaler", StandardScaler()),
        ("model", MLPRegressor(random_state=random_state, max_iter=1000, early_stopping=True))
    ])


In [169]:
x_train_clean, y_train_clean, x_test_clean = remove_outliers(
    x_train, y_train, x_test, contamination=0.047, random_state=42
)

[VarianceThreshold] Removed 4 low-variance features (threshold=1e-07)
[OutlierRemoval] Removed 57 outliers (4.70%)
[Outlier positions] [3, 10, 27, 60, 114, 156, 167, 203, 207, 213, 232, 329, 346, 380, 382, 423, 441, 442, 446, 459, 537, 575, 579, 611, 627, 674, 676, 681, 690, 694, 697, 733, 740, 763, 769, 805, 899, 906, 915, 933, 935, 951, 983, 1015, 1032, 1070, 1071, 1087, 1090, 1097, 1103, 1127, 1136, 1137, 1144, 1152, 1196]


In [170]:
imputation_pipeline = build_imputation_pipeline()

svr_branch = build_svr_branch(random_state=42)

svr_full_pipeline = Pipeline([
    ("imputation", imputation_pipeline),
    ("svr", svr_branch),
])

svr_param_grid = {
    "imputation__knn_imputer__n_neighbors": [2],
    "imputation__knn_imputer__weights": ["distance"],
    "svr__feature_selector__feature_tops": [(200, 175)],
    "svr__feature_selector__rf_n_estimators": [1000],
    "svr__feature_selector__rf_max_depth": [None],
    "svr__feature_selector__rf_min_samples_leaf": [1],
    "svr__feature_selector__rf_max_features": ["sqrt"],
    "svr__model__nu": [0.5],
    "svr__model__kernel": ["rbf"],
    "svr__model__C": [60],
    "svr__model__gamma": ["auto"],
}

svr_grid = GridSearchCV(
    estimator=svr_full_pipeline,
    param_grid=svr_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=2,
)

print("=" * 60)
print("Optimizing SVR Branch")
print("=" * 60)

svr_grid.fit(x_train_clean, y_train_clean)

print(f"\nBest SVR R²: {svr_grid.best_score_:.4f}")
print(f"Best SVR Params: {svr_grid.best_params_}")

Optimizing SVR Branch
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[SpearmanRFSelector] Spearman: Selected 200 features (from 828)
[SpearmanRFSelector] RF: Selected 175 features (from 200)
[SpearmanRFSelector] FINAL: 175 features

Best SVR R²: 0.6756
Best SVR Params: {'imputation__knn_imputer__n_neighbors': 2, 'imputation__knn_imputer__weights': 'distance', 'svr__feature_selector__feature_tops': (200, 175), 'svr__feature_selector__rf_max_depth': None, 'svr__feature_selector__rf_max_features': 'sqrt', 'svr__feature_selector__rf_min_samples_leaf': 1, 'svr__feature_selector__rf_n_estimators': 1000, 'svr__model__C': 60, 'svr__model__gamma': 'auto', 'svr__model__kernel': 'rbf', 'svr__model__nu': 0.5}


In [171]:
imputation_pipeline = build_imputation_pipeline()

hgb_branch = build_hgb_branch(random_state=42)

hgb_full_pipeline = Pipeline([
    ("imputation", imputation_pipeline),
    ("hgb", hgb_branch),
])

hgb_param_grid = {
    "imputation__knn_imputer__n_neighbors": [2],
    "imputation__knn_imputer__weights": ["distance"],

    "hgb__feature_selector__feature_tops":[(200, 167)],
    "hgb__feature_selector__rf_n_estimators": [1000],
    "hgb__feature_selector__rf_max_depth": [None],
    "hgb__feature_selector__rf_min_samples_leaf": [1],
    "hgb__feature_selector__rf_max_features": ["sqrt"],

    "hgb__model__learning_rate": [0.1],
    "hgb__model__max_iter": [200],
    "hgb__model__max_depth": [None],
    "hgb__model__min_samples_leaf": [20],
}

hgb_grid = GridSearchCV(
    estimator=hgb_full_pipeline,
    param_grid=hgb_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=2,
)

print("=" * 60)
print("Optimizing HGB Branch")
print("=" * 60)

hgb_grid.fit(x_train_clean, y_train_clean)

print(f"\nBest HGB R²: {hgb_grid.best_score_:.4f}")
print(f"Best HGB Params: {hgb_grid.best_params_}")

Optimizing HGB Branch
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[SpearmanRFSelector] Spearman: Selected 200 features (from 828)
[SpearmanRFSelector] RF: Selected 167 features (from 200)
[SpearmanRFSelector] FINAL: 167 features

Best HGB R²: 0.6449
Best HGB Params: {'hgb__feature_selector__feature_tops': (200, 167), 'hgb__feature_selector__rf_max_depth': None, 'hgb__feature_selector__rf_max_features': 'sqrt', 'hgb__feature_selector__rf_min_samples_leaf': 1, 'hgb__feature_selector__rf_n_estimators': 1000, 'hgb__model__learning_rate': 0.1, 'hgb__model__max_depth': None, 'hgb__model__max_iter': 200, 'hgb__model__min_samples_leaf': 20, 'imputation__knn_imputer__n_neighbors': 2, 'imputation__knn_imputer__weights': 'distance'}


In [172]:
imputation_pipeline = build_imputation_pipeline()

xgb_branch = build_xgb_branch(random_state=42)

xgb_full_pipeline = Pipeline([
    ("imputation", imputation_pipeline),
    ("xgb", xgb_branch),
])

xgb_param_grid = {
    "imputation__knn_imputer__n_neighbors": [2],
    "imputation__knn_imputer__weights": ["distance"],
    "xgb__feature_selector__feature_tops": [(200, 175)],
    "xgb__feature_selector__rf_n_estimators": [1000],
    "xgb__feature_selector__rf_max_depth": [None],
    "xgb__feature_selector__rf_min_samples_leaf": [1],
    "xgb__feature_selector__rf_max_features": ["sqrt"],
    "xgb__model__n_estimators": [1000],
    "xgb__model__learning_rate": [0.1],
    "xgb__model__max_depth": [3],
    "xgb__model__min_child_weight": [1.2],
    "xgb__model__subsample": [1.0],
    "xgb__model__colsample_bytree": [1.0],
    "xgb__model__gamma": [0],
}

xgb_grid = GridSearchCV(
    estimator=xgb_full_pipeline,
    param_grid=xgb_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=2,
)

print("=" * 60)
print("Optimizing XGBoost Branch")
print("=" * 60)

xgb_grid.fit(x_train_clean, y_train_clean)

print(f"\nBest XGBoost R²: {xgb_grid.best_score_:.4f}")
print(f"Best XGBoost Params: {xgb_grid.best_params_}")

Optimizing XGBoost Branch
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[SpearmanRFSelector] Spearman: Selected 200 features (from 828)
[SpearmanRFSelector] RF: Selected 175 features (from 200)
[SpearmanRFSelector] FINAL: 175 features

Best XGBoost R²: 0.6434
Best XGBoost Params: {'imputation__knn_imputer__n_neighbors': 2, 'imputation__knn_imputer__weights': 'distance', 'xgb__feature_selector__feature_tops': (200, 175), 'xgb__feature_selector__rf_max_depth': None, 'xgb__feature_selector__rf_max_features': 'sqrt', 'xgb__feature_selector__rf_min_samples_leaf': 1, 'xgb__feature_selector__rf_n_estimators': 1000, 'xgb__model__colsample_bytree': 1.0, 'xgb__model__gamma': 0, 'xgb__model__learning_rate': 0.1, 'xgb__model__max_depth': 3, 'xgb__model__min_child_weight': 1.2, 'xgb__model__n_estimators': 1000, 'xgb__model__subsample': 1.0}


In [173]:
imputation_pipeline = build_imputation_pipeline()

cat_branch = build_cat_branch(random_state=42)

cat_full_pipeline = Pipeline([
    ("imputation", imputation_pipeline),
    ("cat", cat_branch),
])

cat_param_grid = {
    "imputation__knn_imputer__n_neighbors": [2],
    "imputation__knn_imputer__weights": ["distance"],

    "cat__feature_selector__feature_tops": [(200, 179)],
    "cat__feature_selector__rf_n_estimators": [1000],
    "cat__feature_selector__rf_max_depth": [None],
    "cat__feature_selector__rf_min_samples_leaf": [1],
    "cat__feature_selector__rf_max_features": ["sqrt"],


    # Number of trees (like n_estimators)
    "cat__model__iterations": [1000],  # Default 1000

    # Learning rate - Lower = need more trees but often better
    "cat__model__learning_rate": [0.03],  # Default is auto-calculated, 0.03 is common

    # Tree depth - Controls model complexity
    "cat__model__depth": [6],  # Default 6

    # L2 regularization on leaf weights
    "cat__model__l2_leaf_reg": [3],  # Default 3.0

    # Minimum number of training samples per leaf
    "cat__model__min_data_in_leaf": [1],  # Default 1

    # Bayesian bootstrap intensity (like subsample)
    "cat__model__bagging_temperature": [1],  # Default 1.0, higher = more aggressive

    # Amount of randomness for scoring splits
    "cat__model__random_strength": [1],  # Default 1.0
}

cat_grid = GridSearchCV(
    estimator=cat_full_pipeline,
    param_grid=cat_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=2,
)

print("=" * 60)
print("Optimizing CatBoost Branch")
print("=" * 60)

cat_grid.fit(x_train_clean, y_train_clean)

print(f"\nBest CatBoost R²: {cat_grid.best_score_:.4f}")
print(f"Best CatBoost Params: {cat_grid.best_params_}")

Optimizing CatBoost Branch
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[SpearmanRFSelector] Spearman: Selected 200 features (from 828)
[SpearmanRFSelector] RF: Selected 179 features (from 200)
[SpearmanRFSelector] FINAL: 179 features

Best CatBoost R²: 0.6552
Best CatBoost Params: {'cat__feature_selector__feature_tops': (200, 179), 'cat__feature_selector__rf_max_depth': None, 'cat__feature_selector__rf_max_features': 'sqrt', 'cat__feature_selector__rf_min_samples_leaf': 1, 'cat__feature_selector__rf_n_estimators': 1000, 'cat__model__bagging_temperature': 1, 'cat__model__depth': 6, 'cat__model__iterations': 1000, 'cat__model__l2_leaf_reg': 3, 'cat__model__learning_rate': 0.03, 'cat__model__min_data_in_leaf': 1, 'cat__model__random_strength': 1, 'imputation__knn_imputer__n_neighbors': 2, 'imputation__knn_imputer__weights': 'distance'}


In [175]:
gpr_kernel = (kernels.ConstantKernel(1.0, (1e-3, 1e3))
              * kernels.RationalQuadratic(length_scale=1.0, alpha=1.0)
              + kernels.WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-6, 1e1)))

# Build GaussianProcess pipeline
imputation_pipeline = build_imputation_pipeline()

gpr_branch = build_gpr_branch(random_state=42)

gpr_full_pipeline = Pipeline([
    ("imputation", imputation_pipeline),
    ("gpr", gpr_branch),
])

gpr_param_grid = {
    "imputation__knn_imputer__n_neighbors": [2],
    "imputation__knn_imputer__weights": ["distance"],
    "gpr__feature_selector__feature_tops": [
        (200, 175),
    ],
    "gpr__feature_selector__rf_n_estimators": [1000],
    "gpr__feature_selector__rf_max_depth": [None],
    "gpr__feature_selector__rf_min_samples_leaf": [1],
    "gpr__feature_selector__rf_max_features": ["sqrt"],

    # Kernel - use the custom one defined above
    "gpr__model__kernel": [gpr_kernel],

    # Noise level added to diagonal of kernel matrix (regularization)
    "gpr__model__alpha": [1e-6],  # Default 1e-10, higher = more regularization

    # Whether to normalize target values (often helps)
    "gpr__model__normalize_y": [True],  # Default False

    # Number of restarts for optimizer (more = better but slower)
    "gpr__model__n_restarts_optimizer": [2],  # Default 0, higher = more robust
}

gpr_grid = GridSearchCV(
    estimator=gpr_full_pipeline,
    param_grid=gpr_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=3,
)

print("=" * 60)
print("Optimizing GaussianProcess Branch (This will be SLOW!)")
print("=" * 60)

gpr_grid.fit(x_train_clean, y_train_clean)

print(f"\nBest GaussianProcess R²: {gpr_grid.best_score_:.4f}")
print(f"Best GaussianProcess Params: {gpr_grid.best_params_}")

Optimizing GaussianProcess Branch (This will be SLOW!)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[SpearmanRFSelector] Spearman: Selected 200 features (from 828)
[SpearmanRFSelector] RF: Selected 175 features (from 200)
[SpearmanRFSelector] FINAL: 175 features


/usr/local/lib/python3.12/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(



Best GaussianProcess R²: 0.6786
Best GaussianProcess Params: {'gpr__feature_selector__feature_tops': (200, 175), 'gpr__feature_selector__rf_max_depth': None, 'gpr__feature_selector__rf_max_features': 'sqrt', 'gpr__feature_selector__rf_min_samples_leaf': 1, 'gpr__feature_selector__rf_n_estimators': 1000, 'gpr__model__alpha': 1e-06, 'gpr__model__kernel': 1**2 * RationalQuadratic(alpha=1, length_scale=1) + WhiteKernel(noise_level=0.001), 'gpr__model__n_restarts_optimizer': 2, 'gpr__model__normalize_y': True, 'imputation__knn_imputer__n_neighbors': 2, 'imputation__knn_imputer__weights': 'distance'}


In [176]:
# SVR
print("\n[1/5] SVR - Generating CV predictions...")
svr_pipeline = svr_grid.best_estimator_
y_svr_cv_pred = cross_val_predict(svr_pipeline, x_train_clean, y_train_clean, cv=5, n_jobs=-1)
print(f"SVR CV predictions shape: {y_svr_cv_pred.shape}")

# HGB
print("\n[2/5] HGB - Generating CV predictions...")
hgb_pipeline = hgb_grid.best_estimator_
y_hgb_cv_pred = cross_val_predict(hgb_pipeline, x_train_clean, y_train_clean, cv=5, n_jobs=-1)
print(f"HGB CV predictions shape: {y_hgb_cv_pred.shape}")

# XGB
print("\n[3/5] XGB - Generating CV predictions...")
xgb_pipeline = xgb_grid.best_estimator_
y_xgb_cv_pred = cross_val_predict(xgb_pipeline, x_train_clean, y_train_clean, cv=5, n_jobs=-1)
print(f"XGB CV predictions shape: {y_xgb_cv_pred.shape}")

# CAT
print("\n[4/5] CAT - Generating CV predictions...")
cat_pipeline = cat_grid.best_estimator_
y_cat_cv_pred = cross_val_predict(cat_pipeline, x_train_clean, y_train_clean, cv=5, n_jobs=-1)
print(f"CAT CV predictions shape: {y_cat_cv_pred.shape}")

# GPR
print("\n[5/5] GPR - Generating CV predictions...")
gpr_pipeline = gpr_grid.best_estimator_
y_gpr_cv_pred = cross_val_predict(gpr_pipeline, x_train_clean, y_train_clean, cv=5, n_jobs=-1)
print(f"GPR CV predictions shape: {y_gpr_cv_pred.shape}")


[1/5] SVR - Generating CV predictions...
SVR CV predictions shape: (1155,)

[2/5] HGB - Generating CV predictions...
HGB CV predictions shape: (1155,)

[3/5] XGB - Generating CV predictions...
XGB CV predictions shape: (1155,)

[4/5] CAT - Generating CV predictions...
CAT CV predictions shape: (1155,)

[5/5] GPR - Generating CV predictions...
GPR CV predictions shape: (1155,)


In [183]:
models_dict = {
    'SVR': y_svr_cv_pred,
    'HGB': y_hgb_cv_pred,
    'XGB': y_xgb_cv_pred,
    'CAT': y_cat_cv_pred,
    'GPR': y_gpr_cv_pred,
}

model_names = list(models_dict.keys())
n_models = len(model_names)

# ========================================================================
# SECTION 1: INDIVIDUAL MODEL PERFORMANCE
# ========================================================================
print("\n" + "=" * 80)
print("SECTION 1: Individual Model Performance")
print("=" * 80)

performance_data = []
for name, preds in models_dict.items():
    r2 = r2_score(y_train_clean, preds)
    rmse = np.sqrt(mean_squared_error(y_train_clean, preds))
    mae = mean_absolute_error(y_train_clean, preds)
    performance_data.append({
        'Model': name,
        'R²': r2,
        'RMSE': rmse,
        'MAE': mae
    })

performance_df = pd.DataFrame(performance_data).sort_values('R²', ascending=False)
print("\n" + performance_df.to_string(index=False))

best_model = performance_df.iloc[0]['Model']
best_r2 = performance_df.iloc[0]['R²']
print(f"\nBest Individual Model: {best_model} (R² = {best_r2:.4f})")

# ========================================================================
# SECTION 2: PREDICTION CORRELATION ANALYSIS
# ========================================================================
print("\n" + "=" * 80)
print("SECTION 2: Prediction Correlation Matrix (Diversity Analysis)")
print("=" * 80)

corr_matrix = np.zeros((n_models, n_models))
for i, name_i in enumerate(model_names):
    for j, name_j in enumerate(model_names):
        corr, _ = pearsonr(models_dict[name_i], models_dict[name_j])
        corr_matrix[i, j] = corr

corr_df = pd.DataFrame(corr_matrix, index=model_names, columns=model_names)
print("\n" + corr_df.to_string())

# Find highly correlated pairs (> 0.90)
print("\nHigh Correlation Pairs (r > 0.90):")
high_corr_pairs = []
for i in range(n_models):
    for j in range(i+1, n_models):
        if corr_matrix[i, j] > 0.90:
            print(f"  {model_names[i]} ↔ {model_names[j]}: r = {corr_matrix[i, j]:.3f} (Very similar!)")
            high_corr_pairs.append((model_names[i], model_names[j], corr_matrix[i, j]))

if not high_corr_pairs:
    print("  None found - Good diversity!")

# Find low correlation pairs (< 0.75)
print("\nLow Correlation Pairs (r < 0.75):")
low_corr_pairs = []
for i in range(n_models):
    for j in range(i+1, n_models):
        if corr_matrix[i, j] < 0.75:
            print(f"  {model_names[i]} ↔ {model_names[j]}: r = {corr_matrix[i, j]:.3f} ✅ (Good diversity!)")
            low_corr_pairs.append((model_names[i], model_names[j], corr_matrix[i, j]))

# ========================================================================
# SECTION 3: ERROR ANALYSIS
# ========================================================================
print("\n" + "=" * 80)
print("SECTION 3: Error Analysis")
print("=" * 80)

# Calculate errors for each model
errors_dict = {}
abs_errors_dict = {}
for name, preds in models_dict.items():
    errors_dict[name] = y_train_clean - preds
    abs_errors_dict[name] = np.abs(y_train_clean - preds)

# Error correlation matrix (do models make errors on same samples?)
error_corr_matrix = np.zeros((n_models, n_models))
for i, name_i in enumerate(model_names):
    for j, name_j in enumerate(model_names):
        corr, _ = pearsonr(abs_errors_dict[name_i], abs_errors_dict[name_j])
        error_corr_matrix[i, j] = corr

error_corr_df = pd.DataFrame(error_corr_matrix, index=model_names, columns=model_names)
print("\nError Correlation Matrix (Do models fail on same samples?):")
print(error_corr_df.to_string())

print("\nHigh Error Correlation Pairs (r > 0.85):")
for i in range(n_models):
    for j in range(i+1, n_models):
        if error_corr_matrix[i, j] > 0.85:
            print(f"  {model_names[i]} ↔ {model_names[j]}: r = {error_corr_matrix[i, j]:.3f} (Fail on same samples)")

# ========================================================================
# SECTION 4: BEST MODEL PER SAMPLE
# ========================================================================
print("\n" + "=" * 80)
print("SECTION 4: Which Model Predicts Each Sample Best?")
print("=" * 80)

# For each sample, find which model has lowest absolute error
all_abs_errors = np.array([abs_errors_dict[name] for name in model_names])
best_model_per_sample = np.argmin(all_abs_errors, axis=0)

# Count how many samples each model predicts best
best_counts = {name: 0 for name in model_names}
for idx in best_model_per_sample:
    best_counts[model_names[idx]] += 1

print("\nSamples Best Predicted by Each Model:")
total_samples = len(y_train_clean)
for name in model_names:
    count = best_counts[name]
    percentage = (count / total_samples) * 100
    print(f"  {name}: {count:4d} samples ({percentage:5.2f}%)")

# ========================================================================
# SECTION 5: COVERAGE ANALYSIS
# ========================================================================
print("\n" + "=" * 80)
print("SECTION 5: Coverage Analysis (Oracle Ensemble)")
print("=" * 80)

# Oracle: For each sample, use the best prediction available
oracle_predictions = np.zeros_like(y_train_clean)
for i in range(len(y_train_clean)):
    best_model_idx = best_model_per_sample[i]
    oracle_predictions[i] = models_dict[model_names[best_model_idx]][i]

oracle_r2 = r2_score(y_train_clean, oracle_predictions)
oracle_rmse = np.sqrt(mean_squared_error(y_train_clean, oracle_predictions))
oracle_mae = mean_absolute_error(y_train_clean, oracle_predictions)

print(f"\nOracle Ensemble Performance (Perfect Model Selection per Sample):")
print(f"  R²:   {oracle_r2:.4f}")
print(f"  RMSE: {oracle_rmse:.4f}")
print(f"  MAE:  {oracle_mae:.4f}")

print(f"\nImprovement vs Best Individual Model:")
print(f"  Best Individual: {best_model} (R² = {best_r2:.4f})")
print(f"  Oracle:          R² = {oracle_r2:.4f}")
print(f"  Potential Gain:  ΔR² = {oracle_r2 - best_r2:.4f}")

threshold = performance_df.iloc[0]['MAE']
print(f"\nCoverage Analysis (threshold = {threshold:.2f} MAE):")

n_models_good_per_sample = np.zeros(total_samples)
for name, abs_errors in abs_errors_dict.items():
    n_models_good_per_sample += (abs_errors <= threshold).astype(int)

coverage_stats = {
    'Covered by 0 models': np.sum(n_models_good_per_sample == 0),
    'Covered by 1 model': np.sum(n_models_good_per_sample == 1),
    'Covered by 2 models': np.sum(n_models_good_per_sample == 2),
    'Covered by 3 models': np.sum(n_models_good_per_sample == 3),
    'Covered by 4 models': np.sum(n_models_good_per_sample == 4),
    'Covered by 5 models': np.sum(n_models_good_per_sample == 5),
}

print("\nSamples Covered by N Models (within threshold):")
for key, count in coverage_stats.items():
    percentage = (count / total_samples) * 100
    print(f"  {key}: {count:4d} samples ({percentage:5.2f}%)")

coverage_any = np.sum(n_models_good_per_sample >= 1)
coverage_any_pct = (coverage_any / total_samples) * 100
print(f"\nTotal samples covered by at least 1 model: {coverage_any} ({coverage_any_pct:.2f}%)")

uncovered = np.sum(n_models_good_per_sample == 0)
uncovered_pct = (uncovered / total_samples) * 100
print(f"Samples where all models struggle: {uncovered} ({uncovered_pct:.2f}%)")


SECTION 1: Individual Model Performance

Model       R²     RMSE      MAE
  GPR 0.678723 5.282552 3.992635
  SVR 0.675945 5.305339 3.988044
  CAT 0.656357 5.463334 4.256988
  HGB 0.645021 5.552710 4.335817
  XGB 0.644223 5.558949 4.321796

Best Individual Model: GPR (R² = 0.6787)

SECTION 2: Prediction Correlation Matrix (Diversity Analysis)

          SVR       HGB       XGB       CAT       GPR
SVR  1.000000  0.896356  0.911060  0.923528  0.978499
HGB  0.896356  1.000000  0.955211  0.961466  0.922791
XGB  0.911060  0.955211  1.000000  0.966840  0.931238
CAT  0.923528  0.961466  0.966840  1.000000  0.945391
GPR  0.978499  0.922791  0.931238  0.945391  1.000000

High Correlation Pairs (r > 0.90):
  SVR ↔ XGB: r = 0.911 (Very similar!)
  SVR ↔ CAT: r = 0.924 (Very similar!)
  SVR ↔ GPR: r = 0.978 (Very similar!)
  HGB ↔ XGB: r = 0.955 (Very similar!)
  HGB ↔ CAT: r = 0.961 (Very similar!)
  HGB ↔ GPR: r = 0.923 (Very similar!)
  XGB ↔ CAT: r = 0.967 (Very similar!)
  XGB ↔ GPR: r = 0.93

In [190]:
def build_stacking_model():
    stacking_model = StackingRegressor(
        estimators=[
            ("gpr", gpr_grid.best_estimator_),
            ("xgb", xgb_grid.best_estimator_),

            ("svr", svr_grid.best_estimator_),
            ("cat", cat_grid.best_estimator_),
            ("hgb", hgb_grid.best_estimator_)
        ],
        final_estimator=LinearRegression(),

    # TODO try, can select the useless models
    # final_estimator=ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000),

    # TODO try out possibly
    # final_estimator=XGBRegressor(
    #     n_estimators=100,
    #     max_depth=3,  # Keep shallow!
    #     learning_rate=0.1,
    #     random_state=42
    # ),
    # TODO try out possibly
    #final_estimator=MLPRegressor(
    #     hidden_layer_sizes=(10, 5),  # Keep small!
    #     alpha=1.0,  # Strong regularization
    #     max_iter=1000,
    #     early_stopping=True,
    #     random_state=42
    # ),

        cv=5,
        n_jobs=-1
    )
    return stacking_model

print("=" * 60)
print("Stacking Ensemble:")
print("=" * 60)

stacking = build_stacking_model()

scores = cross_val_score(
    stacking,
    x_train_clean,
    y_train_clean,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)
print(f"\nStacking CV R²: {scores.mean():.4f} ± {scores.std():.4f}")

Stacking Ensemble:


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 19.5min remaining: 29.2min



Stacking (2 models) CV R²: 0.6940 ± 0.0268


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 19.9min finished


In [191]:
stacking.fit(x_train_clean, y_train_clean)

y_train_pred = stacking.predict(x_train_clean)
train_r2 = r2_score(y_train_clean, y_train_pred)
print(f"Training R²: {train_r2:.4f}")

y_pred = stacking.predict(x_test_clean)

table = pd.DataFrame({'id': np.arange(0, y_pred.shape[0]), 'y': y_pred.flatten()})
table.to_csv('submission.csv', index=False)

Training R²: 0.9918
